In [21]:
import numpy as np

In [22]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [23]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)}은(는) 지원하지 않습니다.')

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0 # 세대 수를 기록하는 변수

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1 # 세대를 기록한다(부모 세대 + 1).

    def cleargrad(self):
        self.grad = None

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop() # 함수를 가져온다.

            # 수정 전: gys = [output.grad for output in f.outputs] # ❶
            gys = [output().grad for output in f.outputs] # ❶
            gxs = f.backward(*gys) # ❷
            if not isinstance(gxs, tuple): # ❸
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs): # ❹
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                    # +=을 쓰지 않는 이유: 값을 덮어쓰는게(인플레이스) 아니라 복사하게끔 + 사용
                    # 인플레이스 연산:  연산에 대한 결괏값을 새로운 변수에 저장하는 것이 아닌
                    # 기존 데이터를 대체하는 것을 의미
                if x.creator is not None:
                    add_func(x.creator) # 수정 전: funcs.append(x.creator)

In [24]:
import weakref

class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)        # ❶ 별표를 붙여 언팩
        if not isinstance(ys, tuple): # ❷ 튜플이 아닌 경우 추가 지원
            ys = (ys,)

        self.generation = max([x.generation for x in inputs])

        outputs = [Variable(as_array(y)) for y in ys]  # Variable 형태로 되돌린다.

        for output in outputs:
            output.set_creator(self) # 출력 변수에 창조자를 설정한다.

        self.inputs  = inputs       # 입력 변수를 기억(보관)한다.
        self.outputs = [weakref.ref(output) for output in outputs]

        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

In [25]:
class Square(Function):
    def forward(self, x):
        return x ** 2

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx

class Exp(Function):
    def forward(self, x):
        return np.exp(x)

    def backward(self, gy):
        x = self.inputs[0].data
        gx = np.exp(x) * gy
        return gx

class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy

In [26]:
def square(x):
    f = Square()
    return f(x)

def exp(x):
    f = Exp()
    return f(x)

def add(x0, x1):
    f = Add()
    return f(x0, x1)

In [27]:
generations = [2, 0, 1, 4, 2]
funcs = []

for g in generations:
    f = Function() # 더미 함수 클래스
    f.generation = g
    funcs.append(f)

[f.generation for f in funcs]

[2, 0, 1, 4, 2]

In [28]:
funcs.sort(key=lambda x : x.generation) # 리스트 정렬

[f.generation for f in funcs]

[0, 1, 2, 2, 4]

In [29]:
f = funcs.pop()
f.generation

4

In [30]:
x = Variable(np.array(2.0))
a = square(x)
y = add(square(a), square(a))
y.backward()

print(y.data)
print(x.grad)

32.0
64.0


In [31]:
for i in range(10):
    x = Variable(np.random.randn(10000)) # 거대한 데이터
    y = square(square(square(x))) # 복잡한 계산을 수행한다.